In [ ]:
%matplotlib notebook
%matplotlib inline

import geopandas as gpd
import matplotlib.pyplot as plt
from cartopy.feature import ShapelyFeature
import cartopy.crs as ccrs
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import pandas as pd
from shapely.geometry import Point, LineString, Polygon
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import io
import zipfile
import json
import math
import os
from osgeo import ogr

plt.ion() # make the plotting interactive

# generate matplotlib handles to create a legend of the features we put in our map.
def generate_handles(labels, colors, edge='k', alpha=1):
    lc = len(colors)  # get the length of the color list
    handles = []
    for i in range(len(labels)):
        handles.append(mpatches.Rectangle((0, 0), 1, 1, facecolor=colors[i % lc], edgecolor=edge, alpha=alpha))
    return handles

# create a scale bar of length 20 km in the upper right corner of the map
# adapted this question: https://stackoverflow.com/q/32333870
# answered by SO user Siyh: https://stackoverflow.com/a/35705477
def scale_bar(ax, location=(0.92, 0.95)):
    llx0, llx1, lly0, lly1 = ax.get_extent(ccrs.PlateCarree())
    sbllx = (llx1 + llx0) / 2
    sblly = lly0 + (lly1 - lly0) * location[1]

    tmc = ccrs.TransverseMercator(sbllx, sblly)
    x0, x1, y0, y1 = ax.get_extent(tmc)
    sbx = x0 + (x1 - x0) * location[0]
    sby = y0 + (y1 - y0) * location[1]

    plt.plot([sbx, sbx - 20000], [sby, sby], color='k', linewidth=9, transform=tmc)
    plt.plot([sbx, sbx - 10000], [sby, sby], color='k', linewidth=6, transform=tmc)
    plt.plot([sbx-10000, sbx - 20000], [sby, sby], color='w', linewidth=6, transform=tmc)

    plt.text(sbx, sby-4500, '20 km', transform=tmc, fontsize=8)
    plt.text(sbx-12500, sby-4500, '10 km', transform=tmc, fontsize=8)
    plt.text(sbx-24500, sby-4500, '0 km', transform=tmc, fontsize=8)

# load the outline of Northern Ireland for a backdrop
outline = gpd.read_file('data/NI_outline.shp')

In [ ]:
# upload data
roads = gpd.read_file('data/NI_roads.shp') 
counties = gpd.read_file('data/Counties.shp')
hospitals = gpd.read_file('data/emergency_care_hospitals.shp')
municipalities = gpd.read_file('data/Municipalities.shp')

In [ ]:
# check coordinate reference system of each dataset
print(roads.crs) 
print(counties.crs)
print(hospitals.crs)
print(municipalities.crs)

In [ ]:
# change reference systems to ITM epsg 2157
roads_itm = roads.to_crs(epsg= 2157) 
counties_itm = counties.to_crs(epsg= 2157)
hospitals_itm = hospitals.to_crs(epsg= 2157)
municipalities_itm = municipalities.to_crs(epsg= 2157)

print(roads_itm.head())

In [ ]:
for i, row in roads_itm.iterrows(): # iterate over each row in the GeoDataFrame
    roads_itm.loc[i, 'Length'] = row['geometry'].length # assign the row's geometry length to a new column, Length
    
print(roads_itm.head()) # print the updated GeoDataFrame to see the changes

In [ ]:
counties_itm #check counties geometry

In [ ]:
# initial view of hosiptal locations
fig, ax = plt.subplots(figsize=(10,10)) 

counties_itm.plot(ax=ax, edgecolor = 'r', facecolor = 'none', column = 'CountyName')
hospitals_itm.plot(ax=ax, markersize = 20, color= 'black', edgecolor = 'black')
roads_itm.plot(ax=ax, cmap='Greys', alpha=.5)

plt.title('Northern Ireland Counties, Ward Population, Roads and Hospitals')

In [ ]:
# uplpad NI wards and water
wards = gpd.read_file('data/NI_Wards.shp')  
water = gpd.read_file('data/Water.shp')

In [ ]:
# change coordinate system to ITM epsg 2157
wards_itm = wards.to_crs(epsg= 2157) 
water_itm = water.to_crs(epsg= 2157)

In [ ]:
print(wards_itm.crs)
print(water_itm.crs)

In [ ]:
wards_itm

In [ ]:
# plot wards showing population
wards_itm.plot(ax=ax, cmap= 'jet', edgecolor= 'black', column= 'Population') 

fig

In [ ]:
# plot ward population with hospital locations
fig, ax = plt.subplots(figsize=(10,10)) 

wards_itm.plot(ax=ax, cmap= 'jet', edgecolor= 'black', column= 'Population')
water_itm.plot(ax=ax, color= 'black')
roads_itm.plot(ax=ax, cmap='Greys', alpha=.5)
hospitals_itm.plot(ax=ax, markersize = 20, edgecolor= 'red', color= 'white')

plt.title('Northern Ireland Ward Population, Roads and Hospitals')

In [ ]:
wards = gpd.read_file('data/NI_Wards.shp')
wards_itm = wards.to_crs(epsg= 2157)

In [ ]:
centroid = wards_itm['geometry'].centroid # produce a centroid for each ward

In [ ]:
centroid.plot(ax = ax, color= 'white') # plot the ward centroids

fig

In [ ]:
centroid

type(centroid) # centroid is  geoseries 

In [ ]:
centroid.to_file("centroid.shp") # save/convert to geodatabase

In [ ]:
centroids = gpd.read_file('centroid.shp')

In [ ]:
type(centroids)

In [ ]:
centroids

In [ ]:
wards_centroid = gpd.sjoin(centroids, wards_itm, how="inner", op='within') # spatial join wards and centroids
wards_centroid

In [ ]:
type(wards_centroid)

In [ ]:
wards_centroid.to_file("ward_cent.shp")

In [ ]:
buffer_20km = hospitals_itm['geometry'].buffer(distance = 20000) # create a 20km/20000m buffer around each hospital
buffer_20km.plot

In [ ]:
# plot hospitals with 20km buffer and wards with ward centroids
fig2, ax1 = plt.subplots(figsize=(10,10)) 

wards_itm.plot(ax=ax1, color= 'none', edgecolor= 'black')
wards_centroid.plot(ax=ax1, color= 'black')
water_itm.plot(ax=ax1, color= 'blue')
buffer_20km.plot(ax=ax1, color= 'none', edgecolor= 'red')
hospitals_itm.plot(ax=ax1, markersize = 20, color= 'red')

plt.title('Northern Ireland Ward Centroids and Hospitals with 20km buffer')

In [ ]:
buffer_20km.to_file("buffer20.shp") # save buffer geoseries and reupload as a geodatabase

In [ ]:
buffer20 = gpd.read_file("buffer20.shp")

In [ ]:
buffer20

In [ ]:
ward_cent = gpd.read_file('ward_cent.shp')

In [ ]:
buff_ward = gpd.sjoin(ward_cent, buffer20, how = 'inner', op = 'intersects') # spatial join of wards inside buffers

In [ ]:
buff_ward

In [ ]:
buffer_overlapped = buffer20[buffer20["FID"].isin([0, 1, 2, 3, 5, 6, 7, 8, 9, 10])] #seprate buffers which overlap (Belfast/Greater Belfast)

In [ ]:
buffer_overlapped

In [ ]:
buffer_overlap_join = buffer_overlapped.geometry.unary_union #join overlapping buffers to produce one polygon

In [ ]:
inside_overlap = ward_cent[ward_cent.geometry.within(buffer_overlap_join)] # show ward centroids inside overlapping buffer zone

inside_overlap

In [ ]:
# show ward centroids inside three remaining separate buffers
buffer_Coleraine = buffer20[buffer20["FID"].isin([4])] 
buffer_LDerry = buffer20[buffer20["FID"].isin([11])]
buffer_Enniskillen = buffer20[buffer20["FID"].isin([12])]

In [ ]:
# count wards inside Coleraine hospital buffer
in_Col = buffer_Coleraine.sindex.query_bulk(ward_cent.geometry, predicate= 'intersects') 
ward_cent['intersects'] = np.isin(np.arange(0, len(ward_cent)), in_Col)

ward_cent['intersects']
ward_cent['intersects'].sum()

In [ ]:
 # count wards inside LDerry hospital buffer
in_Lder = buffer_LDerry.sindex.query_bulk(ward_cent.geometry, predicate= 'intersects')
ward_cent['intersects'] = np.isin(np.arange(0, len(ward_cent)), in_Lder)

ward_cent['intersects'].sum()

In [ ]:
# count wards inside the Enniskillen buffer
in_Enis = buffer_Enniskillen.sindex.query_bulk(ward_cent.geometry, predicate= 'intersects') 
ward_cent['intersects'] = np.isin(np.arange(0, len(ward_cent)), in_Enis)

ward_cent['intersects'].sum()

In [ ]:
# count of wards inside all 20k buffers
wards_in_20 = buffer20.sindex.query_bulk(ward_cent.geometry, predicate = 'intersects') 
ward_cent['intersects'] = np.isin(np.arange(0, len(ward_cent)), wards_in_20)

ward_cent['intersects'].sum()

In [ ]:
580 - 434 # ward centroids not within 20km of an emergency care hospital